# 0. Dependencies

In [1]:
import pandas as pd
from xgboost import XGBRegressor

from numerblox.misc import Key
from numerblox.numerframe import create_numerframe
from numerblox.prediction_loaders import ExamplePredictions
from numerblox.download import NumeraiClassicDownloader
from numerblox.evaluation import NumeraiClassicEvaluator
from numerblox.submission import NumeraiClassicSubmitter

# 1. Download

`NumeraiClassicDownloader` allows you to download training and inference data with a single line of code.

In [2]:
# Download data
downloader = NumeraiClassicDownloader("data")
# Training and validation data
downloader.download_training_data("train_val", version="4.2")

2023-09-28 16:52:45,327 INFO numerapi.utils: target file already exists
2023-09-28 16:52:45,328 INFO numerapi.utils: download complete


2023-09-28 16:52:45,844 INFO numerapi.utils: target file already exists
2023-09-28 16:52:45,845 INFO numerapi.utils: download complete


# 2. Train

We use a custom Pandas DataFrame data structure called `NumerFrame` with `create_numerframe` here to easily parse the Numerai data. The usage of `NumerFrame` is completely optional, but greatly simplify the building of Numerai pipelines and experimentation with Numerai data.

We then fit a simple XGBoost regressor model.

In [3]:
df = create_numerframe("data/train_val/train_int8.parquet")
X, y = df.sample(100).get_feature_target_pair(multi_target=False)
xgb = XGBRegressor()
xgb.fit(X.values, y.values)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

# 3. Evaluate

`NumeraiClassicEvaluator` will calculate all relevant Numerai metrics. 

`ExamplePredictions` is a NumerBlox class that handles downloading of example predictions for you. This object like all other NumerBlox processors can also used end to end in a scikit-learn pipeline.

In [4]:
val_df = create_numerframe("data/train_val/validation_int8.parquet")[:100]
val_df['prediction'] = xgb.predict(val_df.get_feature_data)
val_df['example_preds'] = ExamplePredictions("v4.2/validation_example_preds.parquet").fit_transform(None)['prediction'].values[:100]
evaluator = NumeraiClassicEvaluator()
metrics = evaluator.full_evaluation(val_df, 
                                    example_col="example_preds", 
                                    pred_cols=["prediction"], 
                                    target_col="target")

No existing directory found at 'example_predictions_loader_e7746544-c794-41f1-a700-384a93681309'. Creating directory...


2023-09-28 16:53:09,365 INFO numerapi.utils: starting download
example_predictions_loader_e7746544-c794-41f1-a700-384a93681309/v4.2/validation_example_preds.parquet: 60.9MB [00:02, 21.7MB/s]                            


Path: '/home/clepelaars/numerblox/examples/example_predictions_loader_e7746544-c794-41f1-a700-384a93681309'
Using 'v4.2/features.json/fncv3_features' feature set to calculate FNC metrics.


Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

# 4. Inference

Here again `NumeraiClassicDownloader` and `NumerFrame` are leveraged to simplify inference.

In [5]:
downloader.download_inference_data("current_round", version="4.2")
live_df = create_numerframe(file_path="data/current_round/live_int8.parquet")
live_X, live_y = live_df.get_feature_target_pair(multi_target=False)
preds = xgb.predict(live_X)

2023-09-28 16:53:17,214 INFO numerapi.utils: starting download
data/current_round/live_int8.parquet: 5.90MB [00:00, 16.7MB/s]                            


# 5. Submission

`NumeraiClassicSubmitter` takes care of data integrity checks and submission to Numerai for you. Credentials are conveniently initialized with a `Key` object.

In [6]:
# Submit
NUMERAI_PUBLIC_ID = "YOUR_PUBLIC_ID"
NUMERAI_SECRET_KEY = "YOUR_SECRET_KEY"
key = Key(pub_id=NUMERAI_PUBLIC_ID, secret_key=NUMERAI_SECRET_KEY)
submitter = NumeraiClassicSubmitter(directory_path="sub_current_round", key=key)
# Your prediction file with 'id' as index and defined 'cols' below.
pred_dataf = pd.DataFrame(preds, index=live_df.index, columns=["prediction"])
# Only works with valid key credentials and model_name
# submitter.full_submission(dataf=pred_dataf,
#                           cols="prediction",
#                           file_name="submission.csv",
#                           model_name="MY_MODEL_NAME")

No existing directory found at 'sub_current_round'. Creating directory...


# 6. Clean up environment (optional)

All downloader and submitter have functionality to remove themselver. This is especially convenient if you are running a daily inference pipeline on your server or a cloud VM.

In [7]:
downloader.remove_base_directory()
submitter.remove_base_directory()

Path: '/home/clepelaars/numerblox/examples/data'
Path: '/home/clepelaars/numerblox/examples/sub_current_round'
